# DaZhongDianPing AppSpider
- Author: Yiran Jing

Build DaZhongDianPing AppSpider to collect customer comments of 'Macaroni of Zhang grandpa' chain restaurants

Scraping content:
1. Store location（店铺地址）
2. Score of store （店铺评分）
3. Consumption per person （人均消费）
4. Business Hours （营业时间）
5. Customer comments (网友点评)
6. Recommended dishes (推荐菜)

Reference:
1. https://github.com/Liangchengdeye/DaZhongdianping
2. https://blog.csdn.net/qq_36192201/article/details/89853376

In [133]:
import requests
import json
from lxml import etree
import pandas as pd
import re
import math
import urllib
import urllib.request as urllib2
import xlsxwriter
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from pyquery import PyQuery as pq
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import csv

In [134]:
## 店铺链接输入在这里
url_list_psge = ['/shop/111781377'] # 张爷爷空心面(世纪汇店) 上海

In [135]:
base_url = 'http://www.dianping.com' # 大众点评的网址
url_list = [base_url+url for url in url_list_psge]

In [80]:
soup = BeautifulSoup(html,'lxml')
price

NameError: name 'html' is not defined

In [ ]:
http://www.dianping.com/shop/111781377/review_all

In [127]:

headers = {
    'Host': 'www.dianping.com',
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) App leWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53',
    'Accept-Encoding': 'gzip',
}

def spiderDazhong(ID):
    
        #html = requests.get("http://www.dianping.com/shop/%s/review_all"%(ID), headers=headers)
        html = requests.get("http://www.dianping.com/shop/%s"%(ID), headers=headers)
        doc = pq(html.text)
        
        if doc:
            # 存入csv文件
            out = open('../data/zhangyeye_DaZhongdianping_tags.csv', 'w', newline='',encoding="utf-8")
            # 设定写入模式
            csv_write = csv.writer(out, dialect='excel')
            #shopName = doc("div.body > h1 > a").text()
            #shopurl = "http://www.dianping.com"+doc("div.header-container > h1 > a").attr("href")
            
            
            
            #csv_write.writerow(["店铺名称","店铺网址"])
            #csv_write.writerow([shopName,shopurl])
            
            
            ## tag 收集
            pinglunLi = doc("div.main")#.items()
            
            #print(pinglunLi)
            
            ## Basic infor
            basic_info = doc("div.brief-info")
            print(basic_info)
            
            
            #for data in pinglunLi:
               # print(data.text())
            
            
            
            csv_write.writerow(["用户名", "用户ID链接", "评定星级", "评论描述", "评论详情", "评论时间", "评论商铺", "评论图片"])
            
            
            # 解析评论
            pinglunLi = doc("div.review-items > ul > li").items()
            for data in pinglunLi:
                userName = data("div.main-review > div.dper-info > a").text()
                userID = "http://www.dianping.com"+data("div.main-review > div.dper-info > a").attr("href")
                startShop = str(data("div.review-rank > span").attr("class")).split(" ")[1].replace("sml-str","")
                describeShop = data("div.review-rank > span.score").text()
                pinglunShop = data("div > div.review-words").text().replace("收起评论","").replace(" ","").replace("\n","")
                timeShop = data("div.main-review > div.misc-info.clearfix > span.time").text()
                Shop = data("div.main-review > div.misc-info.clearfix > span.shop").text()
                imgShop = data("div > div.review-pictures > ul > li> a").items()
                imgList = []
                for img in imgShop:
                    imgList.append("http://www.dianping.com"+img.attr("href"))

                # 写入具体内容
                csv_write.writerow([userName,userID,startShop,describeShop,pinglunShop,timeShop,Shop,imgList])
                #print("successful insert csv!")

    
       

In [128]:
# 代表各大商铺ID，可通过商铺列表页回去
listShop = ['111781377']
for shop in listShop:
    spiderDazhong(shop)
    print("successful insert csv!")

<div class="brief-info"> <span title="准四星商户" class="mid-rank-stars mid-str35"/> <span id="reviewCount" class="item"> <d class="num"></d><d class="num"></d><d class="num"></d> 条评论 </span> <div class="star-from-desc J-star-from-desc Hide">星级来自业内综合评估<i class="icon"/></div> <span id="avgPriceTitle" class="item">人均: <d class="num"></d><d class="num"></d> 元</span> <span id="comment_score"> <span class="item">口味: <d class="num"></d>.<d class="num"></d> </span> <span class="item">环境: <d class="num"></d>.<d class="num"></d> </span> <span class="item">服务: <d class="num"></d>.<d class="num"></d> </span> </span> </div> 
successful insert csv!


In [ ]:
## Read in data to check information

In [42]:
data  = pd.read_csv("../data/zhangyeye_DaZhongdianping_comment.csv")
data.head()

,用户名,用户ID链接,评定星级,评论描述,评论详情,评论时间,评论商铺,评论图片
0,找找0516,http://www.dianping.com/member/853068169,50,口味：非常好 环境：很好 服务：很好,道什么，直到世纪汇地商业街业后现这，最门《舌尖2》广告吸入。[环]世纪汇这复古商业街绕，张爷...,2019-07-24 18:09,张爷爷空心面,"['http://www.dianping.com/photos/1666079107', ..."
1,filixpan,http://www.dianping.com/member/23898618,40,口味：好 环境：好 服务：很好,银度好 在世纪汇地下层是吃惜肚放下 一次能吃多了 只能批多次吃一喜沙腐 咬下一口汤算是美香咸...,2019-07-22 23:10,张爷爷空心面,"['http://www.dianping.com/photos/1663272689', ..."
2,qiqishadow,http://www.dianping.com/member/524232,40,口味：很好 环境：很好 服务：很好 人均：35元,张心，我还以为西贝什关呢，多店明亮干净，收银小热情，笑嘻嘻。爆酸汤肥心，事份番腩心，度快。酸...,2019-07-24 15:11,张爷爷空心面,"['http://www.dianping.com/photos/1665783152', ..."
3,风中的少爷,http://www.dianping.com/member/816810005,30,口味：非常好 环境：一般 服务：差 人均：15元,朋友说间吃，坚吃吃，世纪汇B2复古街街真用心，算里面，首先面，想因该物业风给力致烟非恐，投射...,2019-07-21 02:03,张爷爷空心面,"['http://www.dianping.com/photos/1659912030', ..."
4,我叫杨三木,http://www.dianping.com/member/941874865,10,口味：差 环境：差 服务：差,用牌写这么亮！舌尖上美！ 打脸！豆还有牌在门口竖156块！特进去豆！ 等15分钟左右！境油腻...,2019-07-26 23:34,张爷爷空心面,"['http://www.dianping.com/photos/1670004915', ..."


In [24]:
data.shape

(15, 8)

In [131]:

import csv
import requests
from pyquery import PyQuery as pq

headers = {
    'Host': 'www.dianping.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/535.19',
    'Accept-Encoding': 'gzip',
}


def spiderDazhong(ID):
        html = requests.get("http://www.dianping.com/shop/111781377/review_all", headers=headers)
        doc = pq(html.text)
        print(doc)
        
        
        
        print(json.loads(html.text))
        
        
        if doc:
            # 存入csv文件
            out = open('../data/Stu_csv.csv', 'w', newline='',encoding="utf-8")
            # 设定写入模式
            csv_write = csv.writer(out, dialect='excel')
            shopName = doc("div.review-list-header > h1 > a").text()
            shopurl = "http://www.dianping.com"+doc("div.review-list-header > h1 > a").attr("href")
            csv_write.writerow(["店铺名称","店铺网址"])
            csv_write.writerow([shopName,shopurl])
            csv_write.writerow(["用户名", "用户ID链接", "评定星级", "评论描述", "评论详情", "评论时间", "评论商铺", "评论图片"])
            # 解析评论
            pinglunLi = doc("div.reviews-items > ul > li").items()
            for data in pinglunLi:
                userName = data("div.main-review > div.dper-info > a").text()
                userID = "http://www.dianping.com"+data("div.main-review > div.dper-info > a").attr("href")
                startShop = str(data("div.review-rank > span").attr("class")).split(" ")[1].replace("sml-str","")
                describeShop = data("div.review-rank > span.score").text()
                pinglunShop = data("div > div.review-words").text().replace("收起评论","").replace(" ","").replace("\n","")
                timeShop = data("div.main-review > div.misc-info.clearfix > span.time").text()
                Shop = data("div.main-review > div.misc-info.clearfix > span.shop").text()
                imgShop = data("div > div.review-pictures > ul > li> a").items()
                imgList = []
                for img in imgShop:
                    imgList.append("http://www.dianping.com"+img.attr("href"))

                # 写入具体内容
                csv_write.writerow([userName,userID,startShop,describeShop,pinglunShop,timeShop,Shop,imgList])
                print("successful insert csv!")

    #except Exception as e:
       # print("error",str(e))

In [132]:
listShop = ['111781377']
#for shop in listShop:
spiderDazhong(listShop)

<html lang="en">

<head>
    <meta charset="UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
    <meta http-equiv="X-UA-Compatible" content="ie=edge"/>
    <title> 美食, 餐厅餐饮, 团购,生活,优惠券-大众点评网</title>
        <link rel="icon" href="//www.dpfile.com/app/pc-common/dp_favicon.ico" type="image/x-icon"/>
    <link rel="shortcut icon" href="//www.dpfile.com/app/pc-common/dp_favicon.ico" type="image/x-icon"/>
    <link rel="stylesheet" type="text/css" href="//www.dpfile.com/app/pc-common/index.min.css"/>
    <link rel="stylesheet" href="//www.dpfile.com/app/dpindex-new-static/static/new_404_pc.min.css" type="text/css"/>    <!--2. 引入页头模块 js  -->
    <script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '1',  
            'cityCName': '', 
            'cityEnName': '', 
            'pageType': 'index', 
            'userId': '',   
            'userName':'',   
            'dpId': '',      
            'uesrLng': '0.0', 
       

JSONDecodeError: Expecting value: line 1 column 1 (char 0)